In [7]:
!pip install -q accelerate peft bitsandbytes loralib

In [9]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, T5ForConditionalGeneration

model_name = "/home/toibazd/Data/Text2SQL/content/t5-small-finetuned-wikisql"

model = T5ForConditionalGeneration.from_pretrained(model_name, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

In [12]:
model.gradient_checkpointing_enable()

In [13]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [20]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = 20,
    lora_alpha = 1.5,
    lora_dropout=0.1,
    task_type="SEQ_TO_SEQ",
    bias = "none"
)

model = get_peft_model(model, config)

In [21]:
def get_trainable_params(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")
get_trainable_params(model)

trainable params: 737280 || all params: 61243904 || trainable%: 1.203842263223455


In [24]:
from transformers import TrainingArguments
output_dir = '/home/toibazd/Data/Text2SQL/LoRA_adapted_T5'
args = TrainingArguments(
        per_device_train_batch_size = 32, 
        gradient_accumulation_steps = 4,
        warmup_steps=100,  
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir=output_dir)

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.

In [25]:
from transformers import Trainer
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,return_tensors="pt")

trainer = Trainer(
    model = model,
    train_dataset = "Put here train dataset",
    args = args,
    data_collator=data_collator
)

NameError: name 'args' is not defined

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_peft_repo = "psmathur/bloom-7b1-lora-quote-generator"
peft_config = PeftConfig.from_pretrained(hf_peft_repo)
model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Load the finetuned Lora PEFT model
model = PeftModel.from_pretrained(model, hf_peft_repo)

In [ ]:
example_batch = tokenizer("greek, books, wisdom ==>: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**example_batch, max_new_tokens=400)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))